# Redis的发布订阅模式

![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 发布订阅模式的基本语法


### 发布订阅模式是什么

发布订阅模式就像是收音机一样，发布的人不关心有没有人收听，也不关心有多少人收听。他只管发布。接收的人需要订阅这个频道才能收到内容，在订阅之前的内容，接收人都看不到。在他订阅之后，发布者发布的所有信息它都能收听到，直到接收人主动退订为止。发布者发布的所有信息，能同时被所有订阅了这个频道的人接收到。被发布的信息不会储存，发布以后无论有没有人接收，都会直接销毁。

发布订阅模式模式可以用在聊天室，或者其他需要使用到“广播”的场景。

### 发布订阅模式的基本语法

#### 发布消息

```python
client.publish('频道名字', '被广播出来的数据')
```

#### 接收数据

```python

# 允许出现订阅、取消订阅的确认信息
channel = client.pubsub()

# 自动过滤订阅、取消订阅的确认信息
channel = client.pubsub(ignore_subscribe_messages=True)


# 根据完整的频道名字订阅
channel.subscribe('第一个频道', '第二个频道')

# 使用通配符订阅频道
channel.psubscribe('闲话*', '情感*')


# 非阻塞式写法
channel.get_message()   # 接收第一条信息
channel.get_message()  # 接收第二条信息

# 阻塞式写法，如果没有消息，程序会卡在这里，直到有消息进来
for message in p.listen():
    print(message)
```

#### 注册回调函数

```python
def check_at_me(message):
    msg = message['data']
    if 'kingname' in msg:
        print(f'有人在@我，消息内容为：{msg}')
channel = client.pubsub()
channel.subscribe(频道名=回调函数)
channel.get_message()
channel.get_message()
```

#### 取消订阅

```python

# 取消全部频道
channel.unsubscribe()

# 根据完整名称取消特定的订阅
channel.unsubscribe('第一个频道', '第二个频道')

# 根据通配符取消特定的订阅
channel.punsubscribe('闲话*')
```

In [1]:
## 初始化环境

import redis

client = redis.Redis()

## 发布消息

In [2]:
# 注意，此时没有任何人订阅“音乐之声”这个频道，所以没有人能收听到这一条消息
client.publish('音乐之声', '第一首歌：纸短情长')


0

## 订阅频道并接收消息

In [3]:
music = client.pubsub()
music.subscribe('音乐之声')

In [4]:
# 刚刚订阅完成频道，获得的第一条信息是订阅确认信息
music.get_message()  

{'channel': b'\xe9\x9f\xb3\xe4\xb9\x90\xe4\xb9\x8b\xe5\xa3\xb0',
 'data': 1,
 'pattern': None,
 'type': 'subscribe'}

In [5]:
# 此时如果没有新的消息发布，那么获得的信息为None

print(music.get_message())

None


In [6]:
client.publish('音乐之声', '第二首歌：核爆神曲')

1

In [7]:
# 此时这个频道里面已经有消息了，所以可以获取消息了

message = music.get_message()
print('信息的完整内容为：', message)
print(f'信息内容: {message["data"].decode()}')

信息的完整内容为： {'type': 'message', 'pattern': None, 'channel': b'\xe9\x9f\xb3\xe4\xb9\x90\xe4\xb9\x8b\xe5\xa3\xb0', 'data': b'\xe7\xac\xac\xe4\xba\x8c\xe9\xa6\x96\xe6\xad\x8c\xef\xbc\x9a\xe6\xa0\xb8\xe7\x88\x86\xe7\xa5\x9e\xe6\x9b\xb2'}
信息内容: 第二首歌：核爆神曲


In [8]:
message['data'].decode()

'第二首歌：核爆神曲'

In [9]:
## 订阅多个频道

chat = client.pubsub(ignore_subscribe_messages=True)
chat.subscribe('鬼畜专区', '二次元专区')

In [10]:
client.publish('鬼畜专区', 'Are you OK?')
client.publish('二次元专区', '海贼王更新了！！！')
client.publish('二次元专区', '柯南大结局了！！！')
client.publish('鬼畜专区', '金坷垃')

1

In [12]:
# 忽略订阅消息以后，返回的前两条信息为None，这是第一条
message = chat.get_message()
print(message)

None


In [15]:
# 忽略订阅消息以后，返回的前两条信息为None，这是第二条
message = chat.get_message()
print(message)

None


In [16]:
message = chat.get_message()
print(message['data'].decode())

Are you OK?


In [17]:
message = chat.get_message()
print(message['data'].decode())

海贼王更新了！！！


In [18]:
message = chat.get_message()
print(message['data'].decode())

柯南大结局了！！！


In [19]:
message = chat.get_message()
print(message['data'].decode())

金坷垃


## 使用通配符订阅频道



In [20]:
chat.psubscribe('闲话*')

In [21]:
client.publish('闲话1群', '大家随便聊啊')
client.publish('闲话2群', '来来来满上满上')

1

In [22]:
# 第一次返回为None
message = chat.get_message()
print(message)


None


In [23]:
message = chat.get_message()
print(message)
print(message['data'].decode())

{'type': 'pmessage', 'pattern': b'\xe9\x97\xb2\xe8\xaf\x9d*', 'channel': b'\xe9\x97\xb2\xe8\xaf\x9d1\xe7\xbe\xa4', 'data': b'\xe5\xa4\xa7\xe5\xae\xb6\xe9\x9a\x8f\xe4\xbe\xbf\xe8\x81\x8a\xe5\x95\x8a'}
大家随便聊啊


In [24]:
message = chat.get_message()
print(message)
print(message['data'].decode())

{'type': 'pmessage', 'pattern': b'\xe9\x97\xb2\xe8\xaf\x9d*', 'channel': b'\xe9\x97\xb2\xe8\xaf\x9d2\xe7\xbe\xa4', 'data': b'\xe6\x9d\xa5\xe6\x9d\xa5\xe6\x9d\xa5\xe6\xbb\xa1\xe4\xb8\x8a\xe6\xbb\xa1\xe4\xb8\x8a'}
来来来满上满上


In [25]:
client.publish('闲话1群', '今晚谁家吃饭啊')
client.publish('闲话1群', '走我家去')
client.publish('闲话1群', '好啊好啊')

1

In [26]:
for message in chat.listen():
    if not message:
        continue
    print(message['data'].decode())

今晚谁家吃饭啊
走我家去
好啊好啊


KeyboardInterrupt: 

## 注册回调函数

In [27]:
def check_at_me(message):
    msg = message['data'].decode()
    if 'kingname' in msg:
        print(f'有人在@我，消息内容为：{msg}')

In [28]:
emotion = client.pubsub(ignore_subscribe_messages=True)

In [29]:
emotion.subscribe(情感1群=check_at_me)

In [30]:
client.publish('情感1群', '大家好，给大家介绍一下这是我的女朋友')
client.publish('情感1群', '群主@kingname, 快吧这个撒狗粮的踢了')
client.publish('情感1群', '已踢')

1

In [31]:
emotion.get_message()

In [32]:
emotion.get_message()

In [33]:
emotion.get_message()

有人在@我，消息内容为：群主@kingname, 快吧这个撒狗粮的踢了


## 取消订阅



In [34]:
emotion.unsubscribe()

In [35]:
emotion.get_message()

In [36]:
chat.punsubscribe('闲话*')

## 功能测试

In [38]:
client.publish('闲话1群', '行恩和')
client.publish('闲话1群', '嘿嘿嘿')
client.publish('闲话1群', '测试测试')
client.publish('闲话2群', '啊啊')

2

![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)